<a href="https://colab.research.google.com/github/Deep-Learning-Challenge/challenge-notebooks/blob/master/1.Multilayer%20Perceptrons/3.Advanced%20Lessons/1.Save%20Your%20Models%20For%20Later%20With%20Serialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

# Save Your Models For Later With Serialization

Given that deep learning models can take hours, days, and even weeks to train, it is important to know how to save and load them from disk. In this lesson, you will discover how you can save your Keras models to file and load them up again to make predictions. After completing this lesson, you will know:

* How to save and load Keras model weights to HDF5 formatted files.
* How to save and load the Keras model structure to JSON files.

Let's get started.

## Runtime Setup

In [1]:
import sys

dataset_name = "pima-indians-diabetes.data.csv"
if 'google.colab' in sys.modules:
    DATASET = f"https://github.com/Deep-Learning-Challenge/challenge-notebooks/raw/master/datasets/{dataset_name}"
else:
    DATASET = f"../../datasets/{dataset_name}"
    
DATASET

'../../datasets/pima-indians-diabetes.data.csv'

## Tutorial Overview

Keras separates the concerns of saving your model architecture and saving your model weights. Model weights are saved to HDF5 format. This is a grid format that is ideal for storing multi-dimensional arrays of numbers. The model structure can be described and saved (and loaded) using two different formats: JSON and YAML.

Each example will also demonstrate saving and loading your model weights to HDF5 formatted files. The examples will use the same simple network trained on the Pima Indians the onset of diabetes binary classification dataset.

### HDF5 Format

The Hierarchical Data Format, or HDF5 for short, is a flexible data storage format and is convenient for storing large arrays of real values, as we have in the weights of neural networks. You may need to install Python support for the HDF5 file format. You can do this using your preferred Python package management system such as Pip:

`sudo pip install h5py`

## Save Your Neural Network Model to JSON

JSON is a simple file format for describing data hierarchically. Keras provides the ability to describe any model using JSON format with a `to_json()` function. This can be saved to file and later loaded via the `model_from_json()` function that will create a new model from the JSON specification.

The weights are saved directly from the model using the `save_weights()` function and later loaded using the symmetrical `load_weights()` function. The example below trains and evaluates a simple model on the Pima Indians dataset. The model structure is then converted to JSON format and written to `model.json` in the local directory. The network weights are written to `model.h5` in the local directory.

The model and weight data are loaded from the saved files, and a new model is created. It is important to compile the loaded model before it is used. This is so that the model's predictions can use the appropriate, efficient computation from the Keras backend. The model is evaluated in the same way printing the same evaluation score.

In [2]:
import tensorflow as tf

# MLP for Pima Indians Dataset Serialize to JSON and HDF5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
import numpy

# fix random seed for reproducibility
numpy.random.seed(7)

# load pima indians dataset
dataset = numpy.loadtxt(DATASET, delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, Y, epochs=150, batch_size=10, verbose=0)

# evaluate the model
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


2021-10-17 09:50:30.776194: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-17 09:50:30.776430: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-10-17 09:50:40.683507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-17 09:50:40.683622: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-17 09:50:40.683683: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (33d7dde1ac92): /proc/driver/nvidia/version does not exist
2021-10-17 09:50:40.684224: I tensorflow/core/platform/cpu_featu

accuracy: 79.17%


In [3]:

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


Load json and create model

In [5]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile( metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 79.17%


The JSON format of the model looks like the following:

```JSON
{
  "class_name":"Sequential",
  "config":{
    "name":"sequential",
    "layers":[
      {
        "class_name":"InputLayer",
        "config":{
          "batch_input_shape":[
            null,
            8
          ],
          "dtype":"float32",
          "sparse":false,
          "ragged":false,
          "name":"dense_input"
        }
      },
      {
        "class_name":"Dense",
        "config":{
          "name":"dense",
          "trainable":true,
          "batch_input_shape":[
            null,
            8
          ],
          "dtype":"float32",
          "units":12,
          "activation":"relu",
          "use_bias":true,
          "kernel_initializer":{
            "class_name":"RandomUniform",
            "config":{
              "minval":-0.05,
              "maxval":0.05,
              "seed":null
            }
          },
          "bias_initializer":{
            "class_name":"Zeros",
            "config":{

            }
          },
          "kernel_regularizer":null,
          "bias_regularizer":null,
          "activity_regularizer":null,
          "kernel_constraint":null,
          "bias_constraint":null
        }
      },
      {
        "class_name":"Dense",
        "config":{
          "name":"dense_1",
          "trainable":true,
          "dtype":"float32",
          "units":8,
          "activation":"relu",
          "use_bias":true,
          "kernel_initializer":{
            "class_name":"RandomUniform",
            "config":{
              "minval":-0.05,
              "maxval":0.05,
              "seed":null
            }
          },
          "bias_initializer":{
            "class_name":"Zeros",
            "config":{

            }
          },
          "kernel_regularizer":null,
          "bias_regularizer":null,
          "activity_regularizer":null,
          "kernel_constraint":null,
          "bias_constraint":null
        }
      },
      {
        "class_name":"Dense",
        "config":{
          "name":"dense_2",
          "trainable":true,
          "dtype":"float32",
          "units":1,
          "activation":"sigmoid",
          "use_bias":true,
          "kernel_initializer":{
            "class_name":"RandomUniform",
            "config":{
              "minval":-0.05,
              "maxval":0.05,
              "seed":null
            }
          },
          "bias_initializer":{
            "class_name":"Zeros",
            "config":{

            }
          },
          "kernel_regularizer":null,
          "bias_regularizer":null,
          "activity_regularizer":null,
          "kernel_constraint":null,
          "bias_constraint":null
        }
      }
    ]
  },
  "keras_version":"2.4.0",
  "backend":"tensorflow"
}
```

## Summary

Saving and loading models is an important capability for transplanting a deep learning model from research and development to operations. In this lesson, you discovered how to serialize your Keras deep learning models. You learned:

* How to save model weights to HDF5 formatted files and load them again later.
* How to save Keras model definitions to JSON files and load them again.